In [ ]:
#!import BenchmarkAnalysis.dib

In [ ]:
// This only needs to be evaluated when iterating on the Reports-Impl code.

#!import Reports-Impl.dib

# Examples

Note that this is just the state of my working notebook, not a comprehensive set of examples.

In [ ]:
string rootDir = @"C:\home\repro\decommit";
var hugeDM = new DataManager();
foreach (string config in ML("rcbase", "rc1", "alt-draft-log3"))
{
    string configDir = $"gcperfsim_{config}_gc";
    foreach (string benchBase in ML("100mb", "100mb-10mb", "10mb-100mb"))
    {
        foreach (string benchSuffix in ML("", "-low"))
        {
            string bench = benchBase + benchSuffix;
            string etl = $@"{rootDir}\{configDir}\{bench}\{bench}.{config}.0.etl";
            if (!File.Exists(etl))
            {
                Console.WriteLine($"{etl} does not exist");
                continue;
            }
            hugeDM.AddGCTrace(etl, ML("corerun"), config: config, loadMultipleProcesses: false);
        }
    }
}

In [ ]:
hugeDM

In [ ]:
string sds_process_name = "Microsoft.M365.Core.Sds.Service";
string store_worker_process_name = "Microsoft.Exchange.Store.Worker";
var store_work_process_list = ML(sds_process_name, store_worker_process_name);
var sdsDM = DataManager.CreateGCTraces(@"c:\home\repro\notebook-gcapi", store_work_process_list);

(sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.ProcessID,
sdsDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.CommandLine)

In [ ]:
var diffDataManager = DataManager.CreateAspNetData(ML(
    @"C:\home\repro\hc\asp_traceplus3_gc",
    @"C:\home\repro\hc\asp_tp3-m4_gc",
    @"C:\home\repro\hc\asp_slope_gc",
    @"C:\home\repro\hc\asp_evaldecr_gc"
));

In [ ]:
var cardsDM = DataManager.CreateGCTraces(@"c:\home\repro\2401310010004275", pertinentProcesses: ML("EXCEL"));

In [ ]:
var low4DM = DataManager.CreateAspNetData(@"c:\home\repro\hc\asp_v2-fixrearranged-mult-max_gc"
    //, benchmarkFilter: Filter.RE("Stage.*|Json.*");
    //, iterationFilter: IntFilter.Values(1)
);

(low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs.Keys,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max-h4"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.Data.Runs["asp_v2-fixrearranged-mult-max_gc"].Configs["v2-fixrearranged-mult-max"].Benchmarks["Stage2"].Iterations[1].GCSummaryInfo.MaxHeapCount,
 low4DM.GetConfigs(Filter.All, Filter.RE("max")))


In [ ]:
var rc3DataManager = DataManager.CreateAspNetData(@"C:\home\repro\hc\asp_v2-fixrearranged_gc");
rc3DataManager.AddAspNetData(@"C:\home\repro\hc\asp_v2-tune_gc");

rc3DataManager.GetConfigs(Filter.All, Filter.All)

In [ ]:
var net6dm = DataManager.CreateGCTrace(@"C:\home\repro\2046032\Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));
net6dm.AddGCTrace(@"C:\home\repro\2046032\TESTenvCST350 with perf.nettrace", ML("Microsoft.MWC.Workload.OneLake.Service.EntryPoint"));

Charting examples

In [ ]:
ChartBenchmarks(sdsDM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
TableBenchmarks(low4DM, ML(Metrics.B.AveragePercentPauseTimeInGC, Metrics.B.MaxHeapCount)
    //, benchmarkFilter: Filter.Names("JsonMapAction")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    );

In [ ]:
hugeDM.Data.Runs.First().Value.Configs.First().Value.Benchmarks.First().Value.Iterations[0].GCProcessData.GCs

In [ ]:
ChartGCData(hugeDM, ML(Metrics.G.CommittedBeforeInUse, Metrics.G.CommittedBeforeInFree, Metrics.G.CommittedBeforeInGlobalFree, Metrics.G.CommittedBeforeInGlobalDecommit)
  , configNameSimplifier: NameSimplifier.PrefixDashed
  , debug: true
  );

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(low4DM,
    ML(Metrics.B.MaxHeapCount,
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr,
    configFilter: new Filter(excludeRE: "h4")
    //, debug: true
    );
}

In [ ]:
foreach (var xarr in new[] { (XArrangement) XArrangements.Default, XArrangements.Sorted, XArrangements.CombinedSorted })
{
TableBenchmarks(diffDataManager,
    ML((Metrics.B.MaxHeapCount),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Min),
        Metrics.Promote(Metrics.I.MaxHeapCount, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Range),
        Metrics.Promote(Metrics.I.NumberOfHeapCountDirectionChanges, Aggregation.Range)),
    //configNameSimplifier: NameSimplifier.PrefixDashed,
    xArrangement: xarr);
}

In [ ]:
TableIterations(diffDataManager, Metrics.I.MaxHeapCount
    , configFilter: Filter.Names("traceplus3", "tp3-m4")
);

In [ ]:
foreach (var lat in Metrics.I.LatencyMSList)
{
    TableBenchmarks(diffDataManager, Metrics.Promote(lat, Aggregation.Average)
        // , benchmarkFilter: Filter.RE("Stage.*")
        // configNameSimplifier: NameSimplifier.PrefixDashed,
        // types: B_XType.All,
        //configFilter: new Filter(excludeRE: ".*h4")
        );
}

In [ ]:
ChartIterations(diffDataManager, ML(/*Metrics.I.GCScore,*/ Metrics.I.RequestsPerMSec));

In [ ]:
TableGCData(net6dm,
    ML(
        Metrics.G.HeapSizeAfter,
        Metrics.G.HeapSizeBefore,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        Metrics.G.GenLargeSizeBefore,
        Metrics.G.GenLargeSizeAfter,
        Metrics.G.GenLargeObjSizeAfter,
        new Metric<TraceGC>(gc => gc.HeapStats.TotalHeapSize / 1000000, "Total heap size", "MB")
    )
    , textPresenter: TextPresenter.Html
    , configNameSimplifier: new ListSimplifier(("Microsoft.MWC.Workload.OneLake.Service.EntryPoint.exe_WithGCon", "Entry"), ("TESTenvCST350 with perf", "TES"))
    , dataFilter: gc => gc.Number > 300 && gc.Number < 310
    );

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.PauseDuration)
    , benchmarkFilter: Filter.Names("Stage2")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , xMetric: Metrics.X.StartRelativeMSec
    , dataFilter: gc => gc.StartRelativeMSec > 2000
    , iterationFilter: IntFilter.Values(0)
    , xArrangement: XArrangements.Relative
);

In [ ]:
ChartGCData(low4DM
    , metrics: ML(Metrics.G.HctMtcp, Metrics.G.NumHeaps)
    , benchmarkFilter: Filter.RE("Stage2$")
    , configNameSimplifier: NameSimplifier.PrefixDashed
    , iterationFilter: IntFilter.Values(0)
);

In [ ]:
ChartGCData(low4DM, Metrics.G.NumHeaps, configNameSimplifier: NameSimplifier.PrefixDashed, debug: false);

In [ ]:
var low4CompRuns = ML(("v2-fixrearranged-mult-max", "base"), ("v2-fixrearranged-mult-max-h4", "max4"),
    ("v2-fixrearranged-mult-max-svr", "svr"), ("v2-fixrearranged-mult-max-svr4", "svr4"),
    ("v2-fixrearranged-mult-max-mult8", "mult8"), ("v2-fixrearranged-mult-max-mult32", "mult32"),
    ("v2-fixrearranged-mult-max-mult8x10", "m8x10"), ("v2-fixrearranged-mult-max-mult32x10", "m32x10"),
    ("v2-fixrearranged-mult-max-x10", "x10"));

ChartGCData(low4DM, Metrics.G.HctMtcp
    , configFilter: Filter.ExcludeRE("svr")
    , configNameSimplifier: new ListSimplifier(low4CompRuns)
    );

In [ ]:
ChartGCData(
    sdsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    //, benchmarkFilter: Filter.RE("Run32")
    , configNameSimplifier: NameSimplifier.PrefixDashed
);

In [ ]:
ChartGCData(
    cardsDM
    , metrics: ML(Metrics.G.AllocRateMBSec, Metrics.G.PauseDuration)
    , configFilter: Filter.RE("Run32")
);

In [ ]:
ChartGCData(
    cardsDM,
    metrics: ML(Metrics.G.PauseDuration.WithCap(100), Metrics.G.PauseStack.WithCap(100), Metrics.G.PauseFQ, Metrics.G.PauseHandles.WithCap(100), Metrics.G.PauseCards, Metrics.G.Suspend,
        new Metric<TraceGC>(gc => gc.HeapStats.GCHandleCount, "GC Handles", "#"),
        new Metric<TraceGC>(gc => gc.HeapStats.FinalizationPromotedCount, "F promoted", "#"))
    //, dataFilter: gc => gc.Generation == 0
    , configFilter: Filter.RE("Only")
    , xMetric: Metrics.X.GCIndex
    , configNameSimplifier: new ListSimplifier(("2401310010004275", "a"))
    );

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.HeapSizeBefore, benchmarkFilter: Filter.RE("Stage2$"));

In [ ]:
ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , benchmarkFilter: Filter.Names("Fortunes", "FortunesDapper", "JsonHttpsHttpSys", "PlaintextQueryString", "Stage1", "Stage2", "PlaintextMvc")
    , benchmarkMap: x => (x == "Stage1" || x == "Stage2" ? "S1/2" : x));

In [ ]:
var rc3RearrNoBaseRuns = ML(("v2-rc3", "rc3"), ("v2-fixrearranged", "rc3rearr"), ("v2-tune", "rc3tune"));

ChartGCData(rc3DataManager, Metrics.G.NumHeaps
    , configNameSimplifier: new ListSimplifier(rc3RearrNoBaseRuns)
    , benchmarkFilter: new Filter(includeNames: scoutList2)
);

## Debugging

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().Id

In [ ]:
#!about